# Inheritance: For Better or for Worse
We'll cover
- the `super()` function
- the pitfalls of subclassing from built-in types
- Multiple inheritance and method resolution order
- Mixin classes

Discuss: Why is there backlash against the overuse of inheritance? What about multiple inheritance?

## The `super()` Function

In [ ]:
import socketserver
# from the collections module documentation

from collections import OrderedDict

class LastUpdatedOrderedDict(OrderedDict):
    """Store items in the order they were last updated"""

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        self.move_to_end(key)

In [ ]:
# What about code that doesn't use `super()`, but instead calls the method directly on the superclass?

class NotRecommended(OrderedDict):
    """This is a counter example!"""

    def __setitem__(self, key, value):
        OrderedDict.__setitem__(self, key, value)
        self.move_to_end(key)

In [ ]:
# You may see it like this, which is compatible in python 3 (and python 3)

class LastUpdatedOrderedDict(OrderedDict):
    """This code works in python 2 and 3"""

    def __setitem__(self, key, value):
        super(LastUpdatedOrderedDict, self).__setitem__(key, value)
        self.move_to_end(key)

## Subclassing Built-In Types is Tricky

Built-in types, like `list` or `dict` are written in C. This causes problems when you inherit directly from the
built-in type.

Example 14-1: Our `__setitem__` override is ignored by the `__init__` and `__update__` methods of the built-in dict

In [ ]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [ ]:
dd = DoppelDict(one=1)

In [4]:
dd

{'one': 1}

In [5]:
dd["two"] = 2

In [6]:
dd

{'one': 1, 'two': [2, 2]}

In [7]:
dd.update(three=3)

In [8]:
dd

{'one': 1, 'two': [2, 2], 'three': 3}

Example 14-2: The `__getitem__` of `AnswerDict` is bypassed by `dict.update`

In [9]:
class AnswerDict(dict):
    def __getitem__(self, item):
        return 42

In [12]:
ad = AnswerDict(a="foo")
print(ad)

{'a': 'foo'}


In [11]:
ad["a"]

42

In [13]:
ad.get("a")

'foo'

In [14]:
d = {}

In [15]:
d.update(ad)

In [16]:
d

{'a': 'foo'}

The solution? `collections.UserDict`

Example 14-3. `DoppelDict2` and `AnswerDict2` work as expected because they extend `UserDict` and not `dict`

In [17]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [18]:
dd = DoppelDict2(one=1)

In [19]:
dd

{'one': [1, 1]}

In [20]:
dd["two"] = 2

In [21]:
dd

{'one': [1, 1], 'two': [2, 2]}

In [22]:
dd.update(three=3)

In [23]:
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [24]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, item):
        return 42

In [26]:
ad = AnswerDict2(a="foo")
ad

{'a': 'foo'}

In [27]:
ad["a"]

42

In [28]:
d = {}

In [29]:
d.update(ad)

In [30]:
d["a"]

42

In [31]:
d

{'a': 42}

## Multiple Inheritance and Method Resolution Order

If a class has two superclasses, how does Python decide which attribute to use when we call `super().attr` when both
superclasses have an attribute with that name?

This is called the diamond problem.

Example 14-4. `diamond.py`: classes `Leaf`, `A`, `B`, `Root` form the graph.


In [46]:

class Root:  # <1>
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'


class A(Root):  # <2>
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()


class B(Root):  # <3>
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')


class Leaf(A, B):  # <4>
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

Example 14-5. Doctests for calling ping and pong on a `Leaf` object

In [43]:
leaf1 = Leaf()

In [44]:
leaf1.ping()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [ ]:
leaf1.pong()

In [47]:
Leaf.__mro__

(__main__.Leaf, __main__.A, __main__.Root2, __main__.B, __main__.Root, object)

Example 14-6: `diamond2.py`: classes to demonstrate the dynamic nature of `super()`

In [ ]:
class U():  # <2>
    def ping(self):
        print(f'{self}.ping() in U')
        super().ping()  # <3>

class LeafUA(U, A):  # <4>
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()
# end::DIAMOND_CLASSES[]

class LeafAU(A, U):
    def ping(self):
        print(f'{self}.ping() in LeafAU')
        super().ping()

In [ ]:
u = U()

In [ ]:
u.ping()

In [ ]:
leaf2 = LeafUA()

In [ ]:
leaf2.ping()

In [ ]:
LeafUA.__mro__

Example 14-7. MRO of `tkinter.Text`

In [ ]:
def print_mro(cls):
    print(", ".join(c.__name__ for c in cls.__mro__))

import tkinter
print_mro(tkinter.Text)

## Mixin Classes

A mixin class is designed to be subclassed togehter with at least one other class in a multiple inheritance arrangement.

A mixin is not supposed to be the only base class of a concrete class, because it does not provide all the
functionality for a concrete object, but only adds or customizes the behavior of child or sibling classes.

### Case-Insensitive Mappings

Example 14-8. `uppermixin.py`: `UpperCaseMixin` supports case-insensitive mappings

In [ ]:
import collections

def _upper(key):  # <1>
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin:  # <2>
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))

Example 14-9. `uppermixin.py`: two classes that use `UpperCaseMixin`

In [ ]:
class UpperDict(UpperCaseMixin, collections.UserDict):  # <1>
    pass

class UpperCounter(UpperCaseMixin, collections.Counter):  # <2>
    """Specialized 'Counter' that uppercases string keys"""  # <3>

A quick demonstration of `UpperDict`

In [ ]:
d = UpperDict([('a', 'letter A'), (2, 'digit two')])

In [ ]:
list(d.keys())

In [ ]:
d['b'] = 'letter B'

In [ ]:
'b' in d

In [ ]:
d['a'], d.get('B')

In [ ]:
list(d.keys())

A quick demonstration of `UpperCounter`

In [ ]:
c = UpperCounter('BaNanA')
c.most_common()

## Multiple inheritance in the Real World

### ABCs Are Mixins Too

### ThreadingMixIn and ForkingMixIn

The `http.server` package provides `HTTPServer` and `ThreadingHTTPServer` classes. The only difference is:

the threading one uses threads to handle requests by using the `ThreadingMixin`. This is useful to handle web
browsers pre-opening sockets, on which `HTTPServer` would wait indefinitely.

The complete source code for `ThreadingHTTPServer`:

In [ ]:
class ThreadingHTTPServer(socketserver.ThreadingMixIn, HTTPServer):
    daemon_threads = True

Example 14-10. Part of `Lib/sockerserver.py` in python 3.10

In [ ]:
class ThreadingMixIn:
    """Mixin class to handle each request in a new thread."""

    # 8 lines omitted

    def process_request_thread(selfself, request, client_address):
        ... # 6 lines omitted

    def process_request(self, request, client_address):
        ... # 8 lines omitted

    def server_close(self):
        super().server_close()
        self._threads.join()

## Django Generic Views Mixins

See book highlights

## Multiple Inheritance in Tkinter

See book highlights

In [ ]:
import tkinter

print_mro(tkinter.Toplevel)

In [ ]:
print_mro(tkinter.Widget)

In [ ]:
print_mro(tkinter.Button)

In [ ]:
print_mro(tkinter.Entry)

In [ ]:
print_mro(tkinter.Text)

## Coping with Inheritance

### favor object composition over class inheritance

In [ ]:
class GeometryManager():
    def some_method(self):
        print('managing geometry')

In [ ]:
# inheritance
class Widget(GeometryManager):
    def widget_method(self):
        print("making widget")

In [ ]:
# composition
class Widget:
    def __init__(self):
        self.geometry_manager = GeometryManager()

### Understand Why Inheritance is Used in Each Case

The main reasons are:
 - Inheritance of interface creates a subtype, implying an "is-a" relationship. This is best done with ABCs.
 - Inheritance of implementation avoids code duplication by reuse. Mixins can help with this.

### Make Interfaces Explicit with ABCs

### Use Explicit Mixins for Code Reuse

### Provide Aggregate Classes to

From Django's source code, we have:


In [ ]:
class ListView(MultiipleObjectTemplateResponseMixin, BaseListView):
    """
    Render some list of objects, set by `self.model` or `self.queryset`.
    `self.queryset` can actually be an iterable of items, not just a queryset.
    """

Even though the body is empty, the class provides a useful sercice: it brings together a mixin and a base class that
should be used together.

## Subclass Only Classes Designed for Subclassing

Advice: Only create subclasses of classes that were designed to be subclassed.

How do you know if a class was designed to be subclassed?
- documentation
- `@final` - IDEs or type checkers can report misguided attempts to subclass those classes or override those methods

## Avoid Subclassing from Concrete Classes

Why is it dangerous to subclass concrete classes?

All non-leaf classes should be abstract. Said differently, only abstract classes should be subclassed.

# Summary

- `super()` function
    - problems with subclassing built-ins like `list`. Use `UserList` instead
    - Sometimes, it's easier to the appropriate ABC from `collections.abc` and write your own implementations
- Multiple Inheritance
    - `__mro__` addresses the problem of potential naming conflicts of inherited methods
    - `super()` can behave unexpectedly with multiple inheritance. It's designed to support mixin classes
    - We reviewed examples of multiple inheritance and mixins via Django, the python ABCs, and Tkinter.

Note that Go doesn't support inheritance - not even among interfaces. And it's doing just fine.

Perhaps the best advice about inheritance is: avoid it if you can. But often, we don't have a choice: the frameworks
we use impose their own design choices.